Arrumando Modelos

In [ ]:
# -*- coding: utf-8 -*-
import re
from urllib.parse import urlparse, unquote
import pandas as pd
from pathlib import Path

# >>> Caminho de entrada (STRING!), ajuste se necessário
IN_PATH = r"C:\Users\gabriel.vinicius\Documents\Vscode\MicroOnibus\Modelos Onibus.xlsx"
SAIDA   = str(Path(IN_PATH).with_name("Modelos_Onibus_com_modelo.xlsx"))

STOPWORDS = {
    "ano","jm","mlb","cod","codigo","cód","cód.","cod.","ref","novo","usado",
    "rodando","motor","cambio","câmbio","direção","ar","condicionado","codjm"
}
CHASSIS_BRANDS = {
    "mercedes","mercedes-benz","benz","mbenz","mb","volkswagen","vw","volks",
    "volvo","scania","agrale","iveco","man","ford","international"
}
CHASSIS_TOKENS = {"of","oh","oa","ot","onibus","ônibus","urbano","rodoviario","rodoviário"}

YEAR_RE     = re.compile(r"^(19|20)\d{2}$")
COD_RE      = re.compile(r"^cod\d+$", re.IGNORECASE)
PURE_NUM_RE = re.compile(r"^\d+$")

def extract_title_from_url(url: str) -> str:
    if not isinstance(url, str) or not url.strip():
        return ""
    last = unquote(urlparse(url).path.split("/")[-1])
    if "_JM" in last:
        last = last.split("_JM")[0]
    m = re.search(r"MLB-\d+-(.+)", last, flags=re.IGNORECASE)
    title = m.group(1) if m else last
    title = title.strip("-_").replace("-", " ")
    return re.sub(r"\s+", " ", title).strip()

def clean_model_tokens(title: str) -> str:
    if not title:
        return ""
    tokens = title.lower().split()
    cleaned = []
    for tok in tokens:
        if YEAR_RE.match(tok): continue
        if COD_RE.match(tok): continue
        if tok in STOPWORDS: continue
        if tok in CHASSIS_BRANDS: continue
        if tok in CHASSIS_TOKENS: continue
        if PURE_NUM_RE.match(tok): continue
        if tok.startswith("cod") and any(ch.isdigit() for ch in tok): continue
        cleaned.append(tok)
    special = {"vip": "VIP"}
    return " ".join(special.get(t, t.capitalize()) for t in cleaned).strip()

def extract_model(url: str) -> str:
    return clean_model_tokens(extract_title_from_url(url))

# --- Execução ---
# LÊ a planilha a partir do caminho (string)
df = pd.read_excel(IN_PATH)

# Detecta coluna de link (link/url/href), case-insensitive
lower_to_orig = {c.lower(): c for c in df.columns}
for candidate in ("link", "url", "href"):
    if candidate in lower_to_orig:
        LINK_COL = lower_to_orig[candidate]
        break
else:
    raise ValueError("Não encontrei a coluna 'link' (ou 'url'/'href') na planilha.")

# Aplica extrações
df["titulo_bruto"] = df[LINK_COL].astype(str).apply(extract_title_from_url)
df["modelo_extraido"] = df[LINK_COL].astype(str).apply(extract_model)

# Reorganiza colunas
cols = [LINK_COL, "titulo_bruto", "modelo_extraido"] + \
       [c for c in df.columns if c not in {LINK_COL, "titulo_bruto", "modelo_extraido"}]
df = df[cols]

# Salva
df.to_excel(SAIDA, index=False)
print(f"Arquivo salvo em: {SAIDA}")


In [ ]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
from urllib.parse import urlparse, unquote
from pathlib import Path
import unicodedata
from difflib import get_close_matches

# === Caminhos ===
IN_LINKS = r"C:\Users\gabriel.vinicius\Documents\Vscode\MicroOnibus\Modelos Onibus.xlsx"
IN_REF   = r"C:\Users\gabriel.vinicius\Documents\Vscode\MicroOnibus\Onibus.xlsx"
OUT_PATH = str(Path(IN_LINKS).with_name("Modelos_Onibus_normalizado.xlsx"))

# === Utils ===
def strip_accents(s: str) -> str:
    if not isinstance(s, str):
        s = str(s) if s is not None else ""
    nfkd = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in nfkd if not unicodedata.combining(ch))

def normalize_key(s: str) -> str:
    import re
    s = strip_accents(s).lower()
    s = re.sub(r"[^a-z0-9 ]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# === Regras de limpeza ===
STOPWORDS = {
    "ano","jm","mlb","cod","codigo","cód","cód.","cod.","ref","novo","usado",
    "rodando","motor","cambio","câmbio","direção","ar","condicionado","codjm"
}
CHASSIS_BRANDS = {
    "mercedes","mercedes-benz","benz","mbenz","mb","volkswagen","vw","volks",
    "volvo","scania","agrale","iveco","man","ford","international"
}
CHASSIS_TOKENS = {"of","oh","oa","ot","onibus","ônibus","urbano","rodoviario","rodoviário"}
YEAR_RE     = re.compile(r"^(19|20)\d{2}$")
COD_RE      = re.compile(r"^cod\d+$", re.IGNORECASE)
PURE_NUM_RE = re.compile(r"^\d+$")

def extract_title_from_url(url: str) -> str:
    if not isinstance(url, str) or not url.strip():
        return ""
    last = unquote(urlparse(url).path.split("/")[-1])
    if "_JM" in last:
        last = last.split("_JM")[0]
    m = re.search(r"MLB-\d+-(.+)", last, flags=re.IGNORECASE)
    title = m.group(1) if m else last
    title = title.strip("-_").replace("-", " ")
    return re.sub(r"\s+", " ", title).strip()

def clean_model_tokens(title: str) -> str:
    if not title:
        return ""
    tokens = title.lower().split()
    cleaned = []
    for tok in tokens:
        if YEAR_RE.match(tok): continue
        if COD_RE.match(tok): continue
        if tok in STOPWORDS: continue
        if tok in CHASSIS_BRANDS: continue
        if tok in CHASSIS_TOKENS: continue
        if PURE_NUM_RE.match(tok): continue
        if tok.startswith("cod") and any(ch.isdigit() for ch in tok): continue
        cleaned.append(tok)
    special = {"vip": "VIP"}
    return " ".join(special.get(t, t.capitalize()) for t in cleaned).strip()

def extract_model(url: str) -> str:
    return clean_model_tokens(extract_title_from_url(url))

# === Leitura ===
links_df = pd.read_excel(IN_LINKS)
ref_df   = pd.read_excel(IN_REF)

# Detecta coluna de link
lower_to_orig = {c.lower(): c for c in links_df.columns}
for candidate in ("link","url","href"):
    if candidate in lower_to_orig:
        LINK_COL = lower_to_orig[candidate]
        break
else:
    raise ValueError("Não encontrei a coluna 'link' (ou 'url'/'href') na planilha de links.")

# Detecta coluna de modelo na referência (heurística)
prefer = ["modelo_normalizado","modelo referencia","modelo_referencia","modelo",
          "nome_modelo","carroceria","modelo_carroceria","nome"]
prefer_l = [p.lower() for p in prefer]
scores = []
for c in ref_df.columns:
    cl = str(c).lower()
    is_pref = cl in prefer_l
    s = ref_df[c].dropna().astype(str)
    uniq = s.nunique()
    avglen = s.map(len).mean() if len(s)>0 else 0
    scores.append((is_pref, uniq, avglen, c))
scores.sort(key=lambda t: (not t[0], -t[1], -t[2]))
REF_COL = scores[0][3]

# Prepara lista de referência
ref_list = ref_df[REF_COL].dropna().astype(str).drop_duplicates().tolist()
ref_keys = [normalize_key(x) for x in ref_list]
ref_map  = dict(zip(ref_keys, ref_list))

# === Processa ===
out = links_df.copy()
out["titulo_bruto"] = out[LINK_COL].astype(str).apply(extract_title_from_url)
out["modelo_extraido"] = out[LINK_COL].astype(str).apply(extract_model)

def match_reference(s: str, keys, ref_map, cutoff=0.8):
    k = normalize_key(s)
    if not k:
        return ""
    if k in ref_map:
        return ref_map[k]
    hits = get_close_matches(k, keys, n=1, cutoff=cutoff)
    return ref_map[hits[0]] if hits else ""

out["modelo_normalizado"] = out["modelo_extraido"].apply(lambda x: match_reference(x, ref_keys, ref_map, 0.8))
out["modelo_normalizado"] = out.apply(
    lambda r: r["modelo_normalizado"] if isinstance(r["modelo_normalizado"], str) and r["modelo_normalizado"] else r["modelo_extraido"],
    axis=1
)

# Organiza e salva
cols = [LINK_COL, "titulo_bruto", "modelo_extraido", "modelo_normalizado"] + \
       [c for c in out.columns if c not in {LINK_COL,"titulo_bruto","modelo_extraido","modelo_normalizado"}]
out = out[cols]
out.to_excel(OUT_PATH, index=False)
print(f"Arquivo salvo em: {OUT_PATH}\nColuna referência usada: {REF_COL}")


Pegando Anos

In [1]:
import pandas as pd
import re

df = pd.read_excel("Onibus.xlsx")

# tenta achar a coluna 'Tipo' (case-insensitive)
col_tipo = next((c for c in df.columns if str(c).strip().lower() == "tipo"), None)
if col_tipo is None:
    col_tipo = next((c for c in df.columns if "tipo" in str(c).strip().lower()), None)
if col_tipo is None:
    raise ValueError("Coluna 'Tipo' não encontrada.")

def extrair_ano(texto):
    if pd.isna(texto):
        return None
    anos = re.findall(r"(?:19|20)\d{2}", str(texto))
    return int(anos[-1]) if anos else None  # usa o último 19xx/20xx

df["ano 1"] = df[col_tipo].apply(extrair_ano)
df.to_excel("Onibus_com_ano.xlsx", index=False)

Pegando KM

In [3]:
import pandas as pd
import numpy as np

# ===== 1) COLOQUE AQUI O SEU POOL EXATO DE KM =====
km_pool = [
    420000,11,832540,700000,280000,1151,350000,360000,111,195000,358000,38000,151,220000,512000,470000,560000,
    350664,330000,600000,1,250000,200000,300000,500000,397000,296000,383000,400000,585000,450000,542000,380000,
    233064,570000,601000,89000,80000,340000,200,10,150000,172000,890000,77000,5151,345000,428000,645907,445000,
    460000,180000,230000,215000,390000,590000,446500,725000,367522,124,800000,642000,440000,101969,307000,5511,
    205000,10000,160000,50000,61817,900000,277242,650000,540000,520000,650049,369427,536657,530000,124153,341000,
    325000,145000,264000,281000,550000,155,636000,687000,51515,353000,270000,221000,110000,60000,51511,595627,
    240000,214000,6565,685000,515115,2121,740000,1515,5000,260000,985000,90000,295000,670000,435000,122,248000,2,
    490000,680000,290000,710000,128000,183000,454000,11000,398000,304000,591000,554467,604665,515,714000,615000,
    495000,175000,130000,850000,522000,120000,682000,360978,103500,999999,189000,225000,480000,99999,500,621000,
    616000,66669,277000,234788,100,730000,743057,860000,25000,354187,49000,223000,226100,45000,171000,115000,95000,
    830000,141786,4700,288000,750000,639000,124500,83302,102000,412000,382000,365000,4000,331000,465000,475000,
    104000,359500,427000,485000,494000,550,191000,410000,192000,654829,100000,55,530,840000,443000,203900,580000,
    209000,310000,337000,5515,750727,820000,717000,301786,260754,47000,571000,170000,557000,7,618000,1511,265000,
    349000,190000,567000,780000,468000,70000,247000,348000,370000,26400,930000,620000,40000,308000,626000,848000,
    26359,551,768000,405000,633000,980000,878,885000,5454,232000,512,602000,254000,128693,292950,128500,73000,
    493000,478000,375000,368000,277817,15151,154000,320000,500904,849000,11511,1111,653000,992000,193140,255000,
    0,367000,140000,25,35000
]

# ===== 2) FUNÇÃO PRINCIPAL =====
def preencher_km_aleatorio(
    df: pd.DataFrame,
    km_column: str | None = None,
    pool: list[int] = None,
    treat_zero_as_missing: bool = False,
    seed: int | None = None
) -> pd.DataFrame:
    """
    Preenche apenas os valores faltantes na coluna de Km com valores aleatórios do 'pool'.
    - km_column: nome da coluna de quilometragem. Se None, tenta detectar automaticamente.
    - pool: lista de inteiros permitidos para sorteio (usa 'km_pool' se None).
    - treat_zero_as_missing: se True, também trata 0 como faltante.
    - seed: fixa a aleatoriedade (opcional).
    """
    if pool is None:
        pool = km_pool
    if not pool:
        raise ValueError("O 'pool' de quilometragens está vazio.")

    # Detecta a coluna de Km se não foi informada
    if km_column is None:
        km_column = _detectar_coluna_km(df.columns)
        if km_column is None:
            raise ValueError("Coluna 'Km' não encontrada. Informe 'km_column' ou renomeie a coluna.")

    # Semente para reprodutibilidade, se desejar
    rng = np.random.default_rng(seed)

    # Máscara de faltantes: NaN, vazio, 'nan', 'None' (e opcionalmente 0)
    ser = df[km_column]
    mask = ser.isna() | ser.astype(str).str.strip().isin(["", "nan", "None"])

    if treat_zero_as_missing:
        # Converte numericamente só para checar zero (sem alterar o original)
        zero_mask = pd.to_numeric(ser, errors="coerce").fillna(np.nan) == 0
        mask = mask | zero_mask

    n_missing = int(mask.sum())
    if n_missing > 0:
        # Sorteia com reposição a partir do pool
        df.loc[mask, km_column] = rng.choice(pool, size=n_missing, replace=True)

    # Tenta normalizar para numérico (nullable Int64 para manter consistência)
    df[km_column] = pd.to_numeric(df[km_column], errors="coerce").astype("Int64")

    return df

def _detectar_coluna_km(columns) -> str | None:
    # Prioriza match exato "km"
    for c in columns:
        if str(c).strip().lower() == "km":
            return c
    # Depois contém "km" ou "quilometr"
    for c in columns:
        name = str(c).strip().lower()
        if "km" in name or "quilometr" in name:  # pega 'quilometragem', etc.
            return c
    return None

# ===== 3) EXEMPLO DE USO =====
if __name__ == "__main__":
    # Leia sua planilha
    df = pd.read_excel("Faltantes.xlsx")  # ajuste o caminho se necessário

    # Preenche aleatoriamente onde estiver faltando
    df = preencher_km_aleatorio(
        df,
        km_column=None,         # deixa None para auto-detectar ('Km', 'KM', 'quilometragem' etc)
        pool=km_pool,           # seu pool acima
        treat_zero_as_missing=False,  # mude para True se quiser tratar 0 como faltante
        seed=42                 # opcional: fixa o sorteio
    )

    # Salve se quiser
    df.to_excel("Onibus_preenchido.xlsx", index=False)
